In [1]:
import torch
import cv2
import os

/Users/eneskaynakci/.pyenv/versions/3.10.8/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Model
model = torch.hub.load('ultralytics/yolov5', 'custom',path='/Users/eneskaynakci/Github/OTUS22/SİHA Ağırlıklar/UAV-YOLOv5/UAVweights300/best.pt')  # or yolov5n - yolov5x6, custom

Using cache found in /Users/eneskaynakci/.cache/torch/hub/ultralytics_yolov5_master
YOLOv5 🚀 2022-11-17 Python-3.10.8 torch-1.13.0 CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [3]:
# Inference
model.conf = 0.6 # NMS confidence threshold
#model.iou = 0.45  # NMS IoU threshold
#model.agnostic = False  # NMS class-agnostic
#model.multi_label = False  # NMS multiple labels per box
#model.classes = None  # (optional list) filter by class, i.e. = [0, 15, 16] for COCO persons, cats and dogs
#model.max_det = 1000  # maximum number of detections per image
#model.amp = False  # Automatic Mixed Precision (AMP) inference

In [4]:
# Video address
vid_name = '/Users/eneskaynakci/Github/testvids/talonchase.mp4'

# Video Capturer
cap = cv2.VideoCapture(vid_name)

In [5]:
# Options for cv2.rectangle
thickness=1
color=(0,0,255)

# Options for cv2.putText
# font
font = cv2.FONT_HERSHEY_SIMPLEX
# org
org = (50, 50)
# font
fontScale = 0.5
  

In [6]:
import cv2

tracker_types = ['BOOSTING', 'MIL','KCF', 'TLD', 'MEDIANFLOW', 'GOTURN', 'MOSSE', 'CSRT']
tracker_type = tracker_types[7]

if tracker_type == 'BOOSTING':
    tracker = cv2.legacy.TrackerBoosting_create()
if tracker_type == 'MIL':
    tracker = cv2.TrackerMIL_create() 
if tracker_type == 'KCF':
    tracker = cv2.TrackerKCF_create() 
if tracker_type == 'TLD':
    tracker = cv2.legacy.TrackerTLD_create() 
if tracker_type == 'MEDIANFLOW':
    tracker = cv2.legacy.TrackerMedianFlow_create() 
if tracker_type == 'GOTURN':
    tracker = cv2.TrackerGOTURN_create()
if tracker_type == 'MOSSE':
    tracker = cv2.legacy.TrackerMOSSE_create()
if tracker_type == "CSRT":
    tracker = cv2.TrackerCSRT_create()

In [31]:
cnt = 0
flag=1
while(True):
    ret, frame = cap.read()
    if flag:
        results = model(frame)
        df = results.pandas().xyxy[0]
        bbox= [20,10,20,10]
        ret = tracker.init(frame, bbox)
        flag =0
    if ret:
        cnt+=1
        if cnt == 30:
            cnt = 0
            results = model(frame)
            df = results.pandas().xywh[0]
            bbox= (int(df["xmin"]), int(df["ymin"]), int(df["width"]), int(df["height"]))
            ret = tracker.init(frame, bbox)
        ret, bbox = tracker.update(frame)
        # Results
        
        try:
            frame = cv2.rectangle(frame, (int(df["xmin"]),int(df["ymin"])), (int(df["xmax"]),int(df["ymax"])), color, thickness)
            frame = cv2.putText(frame, 'UAV: {}'.format(df["confidence"]), (int(df["xmin"]),int(df["ymin"])), font, 
                   fontScale, color, thickness, cv2.LINE_AA)
        except:
            frame = cv2.putText(frame, 'No detection', org, font, 
                   fontScale, color, thickness, cv2.LINE_AA)

        cv2.imshow("sex", frame)
    else:
        cap.set(cv2.CAP_PROP_POS_FRAMES, 0)
    key = cv2.waitKey(1)
    if key==ord('q'):
        break
cap.release()
cv2.destroyAllWindows()

KeyError: 'xmin'

In [80]:
PATH = "/Users/eneskaynakci/Github/OTUS22/GH010012_Trim/"

for i in os.listdir(PATH):
    frame = cv2.imread(PATH + i)
    frame = cv2.resize(frame, (640,640), interpolation = cv2.INTER_AREA)
    
    results = model(frame)
    # Results
    df = results.pandas().xyxy[0]
    try:
        frame = cv2.rectangle(frame, (int(df["xmin"]),int(df["ymin"])), (int(df["xmax"]),int(df["ymax"])), color, thickness)
        frame = cv2.putText(frame, 'UAV: {}'.format(df["confidence"]), (int(df["xmin"]),int(df["ymin"])), font, 
                   fontScale, color, thickness, cv2.LINE_AA)
    except:
        frame = cv2.putText(frame, 'No detection', org, font, 
                   fontScale, color, thickness, cv2.LINE_AA)

    cv2.imshow("sex", frame)

    key = cv2.waitKey(1)
    if key==ord('q'):
        break
cv2.destroyAllWindows()

In [14]:
cv2.rectangle(frame, (int(df["xmin"]),int(df["xmax"])), (int(df["ymin"]),int(df["ymax"])), color, thickness)

KeyError: 'xmin'